In [8]:
import numpy as np
import requests
from sklearn.linear_model import LinearRegression
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

def get_weather_data():
    api_url = "https://api.open-meteo.com/v1/forecast?latitude=40.7128&longitude=-74.0060&current_weather=true"
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()["current_weather"]
    return None

def main():
    budget = 1_000_000  
    players_to_buy = 2

    train_data = np.array([
        ["Erling Braut Håland", "Attack - Centre-Forward", 39, 38, 200000000, 2],
        ["Bruno Miguel Borges Fernandes", "Midfield - Attacking Midfield", 76, 25, 55000000, 2],
        ["Amad Diallo", "Attack - Right Winger", 6, 1, 25000000, 1],
        ["Kobbie Mainoo", "Midfield - Central Midfield", 10, 0, 55000000, 1],
        ["Alejandro Garnacho Ferreyra", "Attack - Left Winger", 8, 0, 50000000, 1]
    ], dtype=object)
    
    test_data = np.array([
        ["Erling Braut Håland", "Attack - Centre-Forward", 40, 38, 200000000],
        ["Bruno Miguel Borges Fernandes", "Midfield - Attacking Midfield", 80, 28, 60000000],
        ["Amad Diallo", "Attack - Right Winger", 10, 3, 27000000],
        ["Kobbie Mainoo", "Midfield - Central Midfield", 15, 1, 60000000],
        ["Alejandro Garnacho Ferreyra", "Attack - Left Winger", 12, 2, 55000000]
    ], dtype=object)
    
    player_names_train = train_data[:, 0]
    player_stats_train = train_data[:, 2:-2].astype(float)  
    player_values_train = train_data[:, -2].astype(float)
    player_categories_train = train_data[:, -1].astype(int)

    player_names_test = test_data[:, 0]
    player_stats_test = test_data[:, 2:-1].astype(float)
    player_values_test = test_data[:, -1].astype(float)
    
    model = LinearRegression()
    model.fit(player_stats_train, player_values_train)
    
    # Prediksi harga pemain
    predicted_values = model.predict(player_stats_test)
    
    # Neural Network
    y_train = to_categorical(player_categories_train, num_classes=3)
    
    nn_model = Sequential([
        Dense(16, activation='relu', input_shape=(2,)),
        Dense(8, activation='relu'),
        Dense(3, activation='softmax')  # Kategori: Prospect, Value Player, Star
    ])
    
    nn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    nn_model.fit(player_stats_train, y_train, epochs=50, verbose=1)
    
    predicted_categories = np.argmax(nn_model.predict(player_stats_test), axis=1)
    
    # Hitung selisih antara prediksi dan harga pasar sebenarnya
    relative_market_value = [
        {"name": player_names_test[i], "value_gain": player_values_test[i] - predicted_values[i], "price": player_values_test[i], "category": predicted_categories[i]}
        for i in range(len(predicted_values))
    ]
    
    relative_market_value.sort(key=lambda x: x["value_gain"], reverse=True)
    
    selected_players = []
    total_cost = 0
    print(f"Total Budget: {budget}")
    for player in relative_market_value:
        print(f"Evaluating: {player['name']} - Price: {player['price']} - Value Gain: {player['value_gain']}")
        if total_cost + player["price"] <= budget and len(selected_players) < players_to_buy:
            selected_players.append(player)
            total_cost += player["price"]
    
    # Output
    print("\nPemain yang direkomendasikan:")
    if selected_players:
        for p in selected_players:
            category_label = ["Prospect", "Value Player", "Star"][p['category']]
            print(f"{p['name']} - Value Gain: {p['value_gain']} - Price: {p['price']} - Category: {category_label}")
    else:
        print("Tidak ada pemain yang dapat dipilih dalam anggaran yang tersedia.")
    
    print("\nJaringan saraf telah dilatih dan siap memprediksi kategori pemain.")

if __name__ == "__main__":
    main()


Epoch 1/50


C:\Users\Adrianus Ezeekiel\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 619ms/step - accuracy: 0.0000e+00 - loss: 5.2697
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0000e+00 - loss: 5.0502
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0000e+00 - loss: 4.8332
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0000e+00 - loss: 4.6188
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0000e+00 - loss: 4.4070
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0000e+00 - loss: 4.1981
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0000e+00 - loss: 3.9921
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0000e+00 - loss: 3.7893
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0000e+00 - loss: 3.5904
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0000e+00 - loss: 3.4018
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0000e+00 - loss: 3.2176
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step -